Interior-point method for linear programming
The *interior-point* method uses the primal-dual path following algorithm
outlined in [1]_. This algorithm supports sparse constraint matrices and
is typically faster than the simplex methods, especially for large, sparse
problems. Note, however, that the solution returned may be slightly less
accurate than those of the simplex methods and will not, in general,
correspond with a vertex of the polytope defined by the constraints.

References
----------
[1] Andersen, Erling D., and Knud D. Andersen. "The MOSEK interior point
       optimizer for linear programming: an implementation of the
       homogeneous algorithm." High performance optimization. Springer US,
       2000. 197-232.

## Solving Linear Programs using Interior Point Methods

The main idea of the primal-dual method is to solve this set of non-linear equations for a decreasing set of $\mu$. A solution to the equations when $\mu = 0$ is a complementary solution.

\begin{align}
    Ax - b\tau &= 0 \\
    A^T y + z - c\tau &= 0 \\
    -c^Tx + b^Ty - \kappa  &= 0 \\
    Xz - \mu \textbf{e} &= 0 \\
    \tau \kappa - \mu &= 0 \\
\end{align}

Where:
\begin{align}
    x \in \mathbb{R} ^n & \succcurlyeq 0 \\
    z \in \mathbb{R} ^n & \succcurlyeq 0 \\
    \tau \in \mathbb{R} & \geq 0 \\
    \kappa \in \mathbb{R} & \geq 0 \\
    \mu \in \mathbb{R} & \geq 0 \\
    y \in \mathbb{R} ^m & \quad \text{free} \\ 
    A &: \mathbb{R}^n \rightarrow \mathbb{R} ^m \quad \text{A is an $m$ x $n$ matrix}\\
    b &: \mathbb{R}^m \\
    c &: \mathbb{R}^n \\
    x &: \mathbb{R}^n \quad \text{the decision variable we are optimizing for} \\
    X &: \mathbb{R}^n \rightarrow \mathbb{R} ^n \quad \text{X is an $n$ x $n$ diagonal matrix with $x_i$ on the diagonal } \\
    \textbf{e} \in \mathbb{R}^n&: \text{A column vector of 1's} \\
\end{align}


In [27]:
using LinearAlgebra
using Statistics
using Formatting

In [28]:
function get_solver(M)
    (r) -> M \ r
end

get_solver (generic function with 1 method)

In [29]:
function initial_start(m, n)
    x₀ = ones(n)
    y₀ = zeros(m)
    z₀ = ones(n)
    τ₀ = 1
    κ₀ = 1
    return x₀, y₀, z₀, τ₀, κ₀
end

initial_start (generic function with 1 method)

In [30]:
η₁(γ) = (1 - γ)

η₁ (generic function with 1 method)

In [31]:
β = 0.1
get_gamma(z, x) = β * mean(z .* x)

get_gamma (generic function with 1 method)

In [32]:
function get_step(x, dx, z, dz, τ, dtau, κ, dkappa, α₀)
    ix = dx .< 0
    iz = dz .< 0
    
    αx     = α₀ * (any(ix) ? minimum(x[ix] ./ -dx[ix]) : 1)
    αtau   = α₀ * (dtau < 0 ? τ / -dtau : 1)
    αz     = α₀ * (any(iz) ? minimum(z[iz] ./ -dz[iz]) : 1)
    αkappa = α₀ * (dkappa < 0 ? κ / -dkappa : 1)
    
    α = min(1, αx, αtau, αz, αkappa)
    return α
end

get_step (generic function with 1 method)

Assuming an initial solution $(x, \tau, y, z, \kappa)$ where $(x, \tau, z, \kappa) > 0$, then search direction is defined by the following set of linear equations.

\begin{align}
    Ad_x  - bd_{\tau} &= \eta r_p \\
    A^Td_y + d_z - cd_{\tau} &= \eta r_d \\
    -c^Tx + b^Ty - \kappa &= 0 \\
    Xz &= \mu \textbf{e} \\
    \tau \kappa &= \mu \\
\end{align}

and:
\begin{align}
    Zd_x + Xd_z &= -Xz + \gamma \mu \textbf{e} \\
    \kappa d_{\tau} + \tau d_{\kappa} &= -\tau \kappa + \gamma \mu \\
\end{align}

where:
\begin{align}
    r_p &:= b \tau - Ax = -(Ax - b \tau) \\
    r_d &:= c \tau - A^Ty - z = - (A^Ty + z - c \tau)\\
    r_g &:= \kappa + c^Tx - b^Ty = - (b^Ty - c^Tx - \kappa) \\
    \mu &:= \begin{bmatrix} x \\ \tau \end{bmatrix}^T \begin{bmatrix} z \\ \kappa \end{bmatrix}  \div (n+1)\\
\end{align}

$\gamma$ and $\eta$ are two nonnegative parameters. If $\gamma = \eta = 1$ the search direction defined above is equivalent to one newton step.

In each iteration, the Newton equation system below is solved:
 
$$\begin{bmatrix} A & -b &       &     &   &      \\  
                      & -c     & A^T & I &      \\ 
                 -c^T &        & b^T &   & -1   \\
                 Z    &        &     & X &      \\
                      & \kappa &     &   & \tau
\end{bmatrix} \begin{bmatrix}
    d_x \\
    d_{\tau} \\
    d_y \\
    d_z \\
    d_{\kappa}
\end{bmatrix}  = \begin{bmatrix}
    \hat r_p \\
    \hat r_d \\
    \hat r_g \\
    \hat r_{xz} \\
    \hat r_{\tau \kappa}
\end{bmatrix} $$

Where:
\begin{align}
        \hat r_p &:= (1 - \gamma) r_p \\
        \hat r_d &:= (1 - \gamma) r_d \\
        \hat r_g &:= (1 - \gamma) r_g \\ 
        \hat r_{xz} &:= \gamma \mu - xz \\
        \hat r_{\tau \kappa} &:= \gamma \mu - \tau \kappa \\
\end{align}

In [33]:
function get_delta(A, b, c, x, y, z, τ, κ, γ, η)
    m, n = size(A)
    
    rp = b*τ- A*x
    rd = c*τ - A'*y - z
    rg = κ + c'*x - b'*y
    
    μ = (x ⋅ z + τ*κ) / (n + 1)
    
    dinv = x ./ z
    # M is symmetric
    M = A * diagm(dinv) * A'    
    solve = get_solver(M)
    
    α, dx, dz, dtau, dkappa = 0, 0, 0, 0, 0
    dy = missing
    
    for iteration in 1:2
        r⁺p = η(γ) * rp
        r⁺d = η(γ) * rd
        r⁺g = η(γ) * rg

        r⁺xz = γ*μ .- x.*z
        r⁺τκ = γ*μ .- τ*κ
        
        if iteration == 2
            r⁺xz -= dx .* dz
            r⁺τκ -= dtau .* dkappa
        end

        # solve
        p, q = sym_solve(dinv, A, c, b, solve)

        if any(isnan.(p)) || any(isnan.(q))
            throw(DomainError("p, q have NaNs"))
        end

        u, v = sym_solve(dinv, A, r⁺d - (((1) ./ x) .* r⁺xz), r⁺p, solve)

        dtau = (r⁺g + ((1 / τ).* r⁺τκ) - (-c ⋅ u + b ⋅ v)) / (((1) ./ (τ*κ)) + (-c ⋅ p + b ⋅ q))
        dx = u + p .* dtau
        dy = v + q .* dtau
        dz = (1 ./ x) .* (r⁺xz - z .* dx)
        dkappa = 1 ./ τ .* (r⁺τκ - κ .* dtau)
        
        α = get_step(x, dx, z, dz, τ, dtau, κ, dkappa, 1)
        
        β₁ = 0.1
        γ = (1 - α)^2 * min(β₁, 1 - α)
    end
    return dx, dy, dz, dtau, dkappa
end

get_delta (generic function with 1 method)

$$\begin{bmatrix}
    -X^{-1}S & A^T \\
    A & \\
\end{bmatrix} \begin{bmatrix}
    u\\
    v 
\end{bmatrix} = \begin{bmatrix}
    r_1 \\
    r_2
\end{bmatrix}
$$

In [34]:
function sym_solve(dinv, A, r₁, r₂, solve)
    r = r₂ + A * (dinv .* r₁)
    v = solve(r)
    u = dinv .* (A'*v - r₁)
    return u, v
end 

sym_solve (generic function with 1 method)

After the search direction has been computed the variables are updated using
$$(x^+, \tau^+, y^+, s^+, \kappa^+) := (x, \tau, y, s, \kappa) + \alpha (d_x, d_{\tau}, d_y, d_s, d_{\kappa})$$

$\eta = 1 - \gamma$

In [35]:
function do_step(x, y, z, τ, κ, dx, dy, dz, dtau, dkappa, α)
    x = x + α*dx
    y = y + α*dy
    z = z + α*dz
    τ = τ + α*dtau
    κ = κ + α*dkappa
    return x, y, z, τ, κ
end 

do_step (generic function with 1 method)

In [36]:
function get_indicators(A, b, c, x, y, z, τ, κ)
    m, n = size(A)
    x₀, y₀, z₀, τ₀, κ₀ = initial_start(m, n)
    
    rp(x, τ)      = b*τ - A*x
    rd(y, z, τ)   = c*τ - A'*y - z
    rg(x, y, κ)   = κ + c⋅x - b⋅y
    μ(x, τ, z, κ) = (x⋅z + τ*κ) / (n+1)
    
    fx = c ⋅ (x / τ)
    
    rp₀ = rp(x₀, τ₀)
    rd₀ = rd(y₀, z₀, τ₀)
    rg₀ = rg(x₀, y₀, κ₀)
    μ₀  =  μ(x₀, τ₀, z₀, κ₀)
    

    ρp = norm(rp(x, τ)) / max(1, norm(rp₀))
    ρd = norm(rd(y, z, τ)) / max(1, norm(rd₀))
    ρA = norm(c'⋅x - b'⋅y) / (τ + norm(b'⋅y))
    ρg = norm(rg(x, y, κ)) / max(1, norm(rg₀))
    ρμ = μ(x, τ, z, κ) / μ₀
    
    return ρp, ρd, ρA, ρg, ρμ, fx
end
    

get_indicators (generic function with 1 method)

In [37]:
function display_iteration(ρp, ρd, ρg, α, ρμ, fx; header=true)
    if header
        println("Primal Feasibility ",
                "Dual Feasibility   ",
                "Duality Gap        ",
                "Step             ",
                "Path Parameter     ",
                "Objective          ")
    end
    
    s = format("{1:<19.12f}{2:<19.12f}{3:<19.12f}{4:<17.12f}{5:<19.12f}{6:<20.12f}", ρp, ρd, ρg, α, ρμ, fx)
    println(s)
end

display_iteration (generic function with 1 method)

In [38]:
function get_message(status)
    """
    Given problem status code, return a more detailed message.
    Parameters
    ----------
    status : int
        An integer representing the exit status of the optimization::
         0 : Optimization terminated successfully
         1 : Iteration limit reached
         2 : Problem appears to be infeasible
         3 : Problem appears to be unbounded
         4 : Serious numerical difficulties encountered
    Returns
    -------
    message : str
        A string descriptor of the exit status of the optimization.
    """
    messages =
        ["Optimization terminated successfully."
        
         "The iteration limit was reached before the algorithm converged."
        
         "The algorithm terminated successfully and determined that the "
         "problem is infeasible."
        
         "The algorithm terminated successfully and determined that the "
         "problem is unbounded."
         ]
    return messages[status + 1]
end

get_message (generic function with 1 method)

In [39]:
function ip_solve(A, b, c ;α₀=.99995, β=0.1, maxiter=1000, disp=false, tol=1e-8)
    m, n = size(A)
    
    iteration = 0
    status = 0
    
    x, y, z, τ, κ = initial_start(m, n)
    
    ρp, ρd, ρA, ρg, ρμ, fx = get_indicators(A, b, c, x, y, z, τ, κ)
    
    solved = ρp <= tol && ρd <= tol && ρA <= tol
    
    if disp
        display_iteration(ρp, ρd, ρg, NaN, ρμ, fx)
    end
    
    while !solved
        iteration = iteration + 1
        
        γ = 0
        η(γ) = (1 - γ)
            
        # try
        dx, dy, dz, dtau, dkappa = get_delta(A, b, c, x, y, z, τ, κ, γ, η)
    
        α = get_step(x, dx, z, dz, τ, dtau, κ, dkappa, α₀)
        
        x, y, z, τ, κ = do_step(x, y, z, τ, κ, dx, dy, dz, dtau, dkappa, α)
        
        ρp, ρd, ρA, ρg, ρμ, fx = get_indicators(A, b, c, x, y, z, τ, κ)
        
        solved = ρp <= tol && ρd <= tol && ρA <= tol
        
        if disp
            display_iteration(ρp, ρd, ρg, α, ρμ, fx, header=false)
        end
        
        c₁ = ρp < tol && ρd < tol && ρg < tol && τ < tol * max(1, κ)
        c₂ = ρμ < tol && τ < tol * min(1, κ)
        
        if c₁ || c₂
            status = (b' ⋅ y > tol) ? 2 : 3
            break
        elseif iteration >= maxiter
            status = 1
            break
        end
    end
    
    x⁺ = x / τ
    x⁺[x⁺ .< tol] .= 0.0
    z⁺ = z / τ
    y⁺ = y / τ
    return Dict(
        "status"=>status,
        "message"=>get_message(status),
        "x"=>x⁺, 
        "z"=>z⁺, 
        "y"=>y⁺, 
        "nit"=>iteration, 
        "objective"=>c ⋅ x⁺)
end

ip_solve (generic function with 1 method)

In [40]:
function random_problem(m, n)
    A = rand(Float64, (m, m + n))
    x = [4 .+ rand(Float64, m); 1 .+ rand(Float64, n)]
    z = [1 .+ rand(Float64, m); 1 .+ rand(Float64, n)]
    y = rand(Float64, m)
    c = A'*y + z
    b = A*x
    return A, b, c
end

random_problem (generic function with 1 method)

In [41]:
A, b, c = random_problem(200, 200);

In [42]:
@time res = ip_solve(A, b, c, disp=true, tol=1e-8);

Primal Feasibility Dual Feasibility   Duality Gap        Step             Path Parameter     Objective          
1.000000000000     1.000000000000     1.000000000000     NaN              1.000000000000     19619.546406586251  
0.217770449996     0.217770449996     0.217770449996     0.796242788700   0.217770449997     40587.988834993055  
0.060750291515     0.060750291515     0.060737357472     0.737056794635   0.060661266841     52217.927388712342  
0.023247870334     0.023247870334     0.023224375244     0.637992867151   0.023149267569     56002.372370821933  
0.008473066772     0.008473066772     0.008452896432     0.648526217210   0.008413759036     57687.733592616627  
0.002604898234     0.002604898234     0.002597350602     0.708165194052   0.002602635462     58376.379887728443  
0.000949609324     0.000949609324     0.000946858835     0.645341768797   0.000952750255     58570.944751614064  
0.000381712139     0.000381712139     0.000380606078     0.610901158158   0.000383577479 

In [43]:
res

Dict{String, Any} with 7 entries:
  "nit"       => 12
  "status"    => 0
  "message"   => "Optimization terminated successfully."
  "x"         => [7.46029, 8.68788, 4.6637, 4.03119, 2.17568, 0.0, 3.96394, 1.8…
  "objective" => 58682.6
  "z"         => [1.01508e-10, 7.64187e-11, 1.6642e-10, 1.96102e-10, 4.62408e-1…
  "y"         => [0.225053, 0.107913, 0.710078, 0.0676229, 0.955597, 0.699809, …

In [44]:
sum(A' * get(res, "y", nothing) + get(res, "z", nothing) - c)

-5.502055358874713e-6

In [45]:
sum(A * get(res, "x", nothing) - b)

-7.36327815502591e-5

In [47]:
using Cbc
using JuMP

In [48]:
m = Model(Cbc.Optimizer);

In [49]:
_, n = size(A)

(200, 400)

In [50]:
@variable(m, x[1:n]);

In [51]:
@objective(m, Min, c ⋅ x);

In [52]:
@constraint(m, c₁, (A*x) .== b)
@constraint(m, c₂, x .>= 0);

In [53]:
optimize!(m)

Presolve 200 (-400) rows, 400 (0) columns and 80000 (-400) elements
0  Obj 0 Primal inf 5300741.8 (200)
31  Obj 45847.826 Primal inf 30316458 (196)
83  Obj 53539.751 Primal inf 2340840.5 (157)
126  Obj 55811.477 Primal inf 962585.1 (142)
169  Obj 56770.95 Primal inf 2672323.6 (141)
200  Obj 57261.558 Primal inf 1904078.9 (124)
242  Obj 57734.536 Primal inf 657372.54 (112)
286  Obj 58166.058 Primal inf 84245.99 (95)
317  Obj 58313.826 Primal inf 88982.178 (98)
355  Obj 58528.164 Primal inf 16541.057 (98)
398  Obj 58620.818 Primal inf 14652.96 (96)
442  Obj 58667.88 Primal inf 1369.7812 (79)
488  Obj 58679.943 Primal inf 1310.5288 (73)
534  Obj 58682.566
Optimal - objective value 58682.566
After Postsolve, objective 58682.566, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 58682.56648 - 534 iterations time 0.342, Presolve 0.01


In [54]:
objective_value(m)

58682.5664758392

In [55]:
sum(A * (value.(x)) - b)

2.6147972675971687e-12

In [56]:
sum(A * get(res, "x", 0) - b)

-7.36327815502591e-5

In [57]:
objective_value(m) - get(res, "objective", Inf)

3.594792360672727e-5

In [58]:
value.(x)

400-element Vector{Float64}:
  7.460292229237452
  8.68787576670859
  4.6637044151631
  4.031187211901542
  2.175681810959172
  0.0
  3.9639370306622626
  1.835790772283989
  0.0
  0.0
  0.0
  0.0
  1.3051002683103725
  ⋮
  8.66239052545211
  0.0
  2.550116424517169
  3.775352269927083
  0.0
  0.0
  1.4840818291199445
  2.1671105362596603
 14.048642510338114
  0.0
  0.0
 15.036671909701367